In [ ]:
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.time import Time
from astropy.timeseries import TimeSeries

from matplotlib import pyplot as plt

from stellarphot.differential_photometry import vsx_mags


In [ ]:
your_photometry_file = 'v0553 her-2022-06-03.csv'
comp_stars_file = 'apass-V0533-Her.csv'
name_of_variable = 'v0533 her'

# Our filter names vs filter names in comparison star table
filter_mapping = dict(
    ip='SI',
    B='B'
)

In [ ]:
var_coord = SkyCoord.from_name(name_of_variable)

vc = dict(coords=var_coord)

In [ ]:
your_photometry = Table.read(your_photometry_file)
your_photometry['RA'].unit = 'degree'
your_photometry['Dec'].unit = 'degree'

your_photometry['band'] = your_photometry['filter']

comp_stars = Table.read(comp_stars_file)
comp_stars['RAJ2000'].unit = 'degree'
comp_stars['DEJ2000'].unit = 'degree'



## The cell below does just one filter...

In [ ]:
this_filter = 'ip'

this_phot = your_photometry[your_photometry['band'] == this_filter]

this_phot_grp = this_phot.group_by('BJD')

times = []
cal_mag = []
cal_mag_err = []
for time, rows in zip(this_phot_grp.groups.keys, this_phot_grp.groups):
    mag, err = vsx_mags.calc_vmag(vc, rows, comp_stars, band=filter_mapping[this_filter], star_data_mag_column=f'mag_inst_{this_filter}')
    cal_mag.append(mag)
    cal_mag_err.append(err)
    times.append(time[0])

## Sample plot....

In [ ]:
plt.errorbar(times, cal_mag, yerr=cal_mag_err, fmt='s')
plt.grid()